In [9]:
import joblib
import json
import os

import mlflow
import pandas as pd

In [16]:
# Loading the trained model
with open("model/fitted_model_cbr_l.pkl", "rb") as fd:
    model = joblib.load(fd)  # Эта строка внутри блока with (имеет отступ)

# Retrieving Catboost model parameters (без отступа, так как это уже вне блока with)
model_params = model["model"].get_params()

In [17]:
model_params

{'loss_function': 'RMSE',
 'random_seed': 42,
 'verbose': 100,
 'cat_features': [11, 12, 13]}

In [18]:
with open("metrics/eval_results_cbr_l.json") as json_file:
        metrics = json.load(json_file)

In [19]:
metrics

{'r2_score': 0.7007744928684252,
 'rmse': 2462549.744002088,
 'mae': 1873492.8414151592,
 'mape': 1.470389120444638}